In [ ]:
import time
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


dataset = load_dataset("csv", data_files={"train": "ISEAR_train.csv", "test": "ISEAR_test.csv"})


unique_emotions = sorted(set(dataset["train"]["emotion"]))
emotion2label = {emotion: idx for idx, emotion in enumerate(unique_emotions)}
num_labels = len(emotion2label)

def map_emotion_to_label(example):
    example["label"] = emotion2label[example["emotion"]]
    return example

dataset = dataset.map(map_emotion_to_label)


bert_model_name = "bert-base-uncased"
tokenizer_bert = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(
    bert_model_name,
    num_labels=num_labels,
    hidden_dropout_prob=0.3,  #  dropout pre regularizáciu
    attention_probs_dropout_prob=0.3
)


def tokenize_bert(examples):
    return tokenizer_bert(examples["text"], truncation=True, padding="max_length", max_length=128)

dataset = dataset.map(tokenize_bert, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


bert_training_args = TrainingArguments(
    output_dir="./bert_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,  # Ponechanie 10 epoch
    weight_decay=0.1,  # Zvýšenie weight decay pre silnejšiu regularizáciu
    load_best_model_at_end=True,  # Načítanie najlepšieho modelu na konci
    metric_for_best_model="eval_loss",  # Metrika pre výber najlepšieho modelu
    greater_is_better=False,  # Nižšia validačná strata je lepšia
)


bert_trainer = Trainer(
    model=bert_model,
    args=bert_training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Pridanie early stopping
)



start_time = time.time()

bert_trainer.train()

end_time = time.time()
training_time = end_time - start_time

print(f"BERT training complete. Training took {training_time:.2f} seconds.")


print("Evaluating BERT model...")
bert_results = bert_trainer.evaluate()
print(" Výsledky vyhodnotenia BERT:")
print(bert_results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/jovyan/.local/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training BERT model...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.053887,0.644909,0.661051,0.644909,0.642368
2,1.480700,0.905718,0.686031,0.690914,0.686031,0.680204
3,0.935100,0.868984,0.704961,0.712546,0.704961,0.705157
4,0.788700,0.854059,0.716710,0.717210,0.716710,0.715636
5,0.788700,0.853042,0.727807,0.734472,0.727807,0.730089
6,0.665900,0.893615,0.729112,0.729014,0.729112,0.728635
7,0.577400,0.891355,0.730418,0.735445,0.730418,0.731262
8,0.509500,0.920434,0.724543,0.725717,0.724543,0.724610


BERT training complete. Training took 354.23 seconds.
Evaluating BERT model...


📊 Výsledky vyhodnotenia BERT:
{'eval_loss': 0.8530417084693909, 'eval_accuracy': 0.7278067885117493, 'eval_precision': 0.7344717024562537, 'eval_recall': 0.7278067885117493, 'eval_f1': 0.7300892522799318, 'eval_runtime': 2.6965, 'eval_samples_per_second': 568.139, 'eval_steps_per_second': 35.601, 'epoch': 8.0}
